In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Загрузка данных

In [2]:
df = pd.read_csv('stratification_task_data_public.csv')
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,0.869,30,33.8,0,1,0.2,1992,1,1,1,1903
1,0.759,27,21.7,2,0,3.5,1995,1,1,2,1313
2,0.456,29,37.6,2,0,3.1,1993,0,0,0,1484
3,0.060,35,27.5,2,0,4.7,1988,0,0,1,1188
4,0.939,19,30.7,0,0,3.6,2003,1,1,2,842


## Вычисление характеристик стратификации

Напишем функцию для вычисления стратифицированной дисперсии и минимальной доли страт.

In [3]:
def calc_strat_params(df):
    """Вычисляет стратифицированную дисперсию и минимальную долю страт."""
    strat_vars = df.groupby('strat')['y'].var()
    weights = df['strat'].value_counts(normalize=True)
    stratified_var = (strat_vars * weights).sum()
    min_part = df['strat'].value_counts(normalize=True).min()
    return stratified_var, min_part

def print_strat_params(df):
    stratified_var, min_part = calc_strat_params(df)
    print(f'var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

Посчитаем дисперсию без разбиения на страты, чтобы знать с чем сравниваться в дальнейшем.

In [4]:
df['strat'] = 0
print_strat_params(df)

var=66078, min_part=100.00%


## Стратегия 1. Признак в качестве страты

У нас есть данные по 10 признакам. Попробуем взять в качестве страты каждый из них.

In [5]:
for feature in [f'x{x}' for x in range(1, 11)]:
    print(f'{feature:<5}', end='')
    df['strat'] = df[feature]
    print_strat_params(df)

x1   var=66234, min_part=0.02%
x2   var=47232, min_part=0.01%
x3   var=64391, min_part=0.01%
x4   var=66054, min_part=32.61%
x5   var=66022, min_part=40.74%
x6   var=65714, min_part=0.01%
x7   var=48055, min_part=0.01%
x8   var=64859, min_part=34.95%
x9   var=66051, min_part=39.70%
x10  var=63756, min_part=0.07%


Изначальная дисперсия была равна 66078. Все варианты кроме x1 снизили дисперсию.

Сильнее всего дисперсия снизилась при стратификации по признаку x2, но доля минимальной страты меньше 5%.

Лучший результат с учётом ограничения на размер страт: **x8 var=64859, min_part=34.9%**

## Стратегия 2. Объединение страт

При стратификации по признаку x10 были страты размером меньше 5%. Объединим несколько страт в одну, чтобы минимальный размер страт был не меньше 5%.

Чем сильнее отличаются средние значения метрики у страт, тем сильнее понижается дисперсия. Поэтому логично объединять страты с похожими средними значениями.

Для стратификации по признаку x10 посчитаем доли страт и средние значения метрики.

In [6]:
df_agg = df.groupby('x10')[['y']].agg(['count', 'mean'])
df_agg.columns = ['count', 'mean']
df_agg['part'] = (df_agg['count'] / len(df)).round(3)
df_agg['mean'] = df_agg['mean'].round(1)
df_agg

,count,mean,part
x10,,,
0,1362,1311.5,0.136
1,2701,1310.8,0.270
2,2702,1391.6,0.270
3,1825,1411.9,0.182
4,895,1427.0,0.090
5,346,1428.5,0.035
6,132,1425.7,0.013
7,30,1526.9,0.003
8,7,1402.3,0.001


Значения признака 5, 6, 7 и 8 имеют наибольшие средние значения метрики. Доля каждого из этих значений меньше 5%, объединим их в одну страту.

In [7]:
df['strat'] = [min(v, 5) for v in df['x10']]
print_strat_params(df)

var=63761, min_part=5.15%


Мы добились минимально доли страт не меньше 5% и нового минимального значения стратифицированной дисперсии.

**min(x10, 5) var=63761, min_part=5.1%**

## Стратегия 3. Пересечение страт

У нас есть разные способы разбиения на страты. Возьмём несколько способов и на их основе создадим новый, который будет состоять из всех возможных комбинаций других страт.

Например, возьмём страты на основе признаков `x8` и `x9`:

- страта 1: `x8=0, x9=0`
- страта 2: `x8=0, x9=1`
- страта 3: `x8=1, x9=0`
- страта 4: `x8=1, x9=1`

In [8]:
df['strat'] = [' '.join(values) for values in df[['x8', 'x9']].astype(str).values]
print_strat_params(df)

var=64832, min_part=14.17%


Значения дисперсий при стратификации по одному признаку:

- x8   var=64859, min_part=34.9%
- x9   var=66051, min_part=39.7%

Дисперсия пересечения признаков стала ниже.

**(x8+x9) var=64832, min_part=14.2%**

## Три стратегии

Используя описанные подходы можно создавать интерпретируемые разбиения на страты, которые будут снижать дисперсию и повышать чувствтительность А/Б тестов.